<a href="https://colab.research.google.com/github/akutayaydin/Magnimind-5.1-DeepLearning/blob/main/0_WarmUp1CNNExample1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN Example 1
For this example, we have images of cars and flowers, which have been divided into training and testing sets, and we have to build a CNN that identifies whether an image is a car or a flower.

### Step 1: Import the numpy library and the necessary Keras libraries and classes

In [1]:
# Import the Libraries
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np
from tensorflow import random

### Step 2: Now, set a seed and initiate the model with the `Sequential` class

In [2]:
#set a seed
seed = 1
np.random.seed(seed)
random.set_seed(seed)

# Initialising the CNN
classifier = Sequential()

### Step 3: Add the first layer of the CNN, set the input shape to (64, 64, 3), the dimension of each image, and set the activation function as a ReLU:

In [3]:
classifier.add(Conv2D(10,kernel_size =2, activation = 'relu',input_shape = (64,64,3)))

### Step 4: Now, add the pooling layer with the image size as 2x2

In [4]:
classifier.add(MaxPool2D(pool_size =(2,2)))

### Step 5: Flatten the output of the pooling layer by adding a flattening layer to the CNN model:

In [5]:
classifier.add(Flatten())

### Step 6: Add the first Dense layer of the MLP. 
Here, 128 is the output of the number of nodes. As a good practice, 128 is good to get started. activation is relu. As a good practice, the power of two is preferred

In [6]:
classifier.add(Dense(128,activation = 'relu'))

### Step 7: Add the output layer of the MLP.
This is a binary classification problem, so the size is 1 and the activation is `sigmoid`:

In [7]:
classifier.add(Dense(1,activation = 'sigmoid'))

### Step 8: Compile the network
Use an adam optimizer and compute the accuracy during the training process 

In [19]:
classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Step 9: Create training and test data generators. 
- Rescale the training and test images by `1/255` so that all the values are between `0` and `1`.
- Set these parameters for the training data generators only 
 - `shear_range=0.2`, `zoom_range=0.2`, and `horizontal_flip=True`
 
 - https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html


In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
mypath='/content/gdrive/My Drive/Google Colab Folder/'

In [11]:
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.utils import array_to_img, img_to_array, load_img

In [17]:
#Define batch size here
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale = 1. /255,
    shear_range =0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale = 1. /255)

#Train data generation starts here
train_generator = train_datagen.flow_from_directory(
    #str(mypath)+'train', # traget directory
    directory = '/content/gdrive/My Drive/Google Colab Folder/train',
    target_size = (64,64), #resize all images
    batch_size = batch_size,
    class_mode = 'binary'
)

#Test data generation starts here
test_generator = test_datagen.flow_from_directory(
    #str(mypath)+'test',
    directory = '/content/gdrive/My Drive/Google Colab Folder/test',
    target_size = (64,64),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 1569 images belonging to 2 classes.
Found 387 images belonging to 2 classes.


In [21]:
classifier.fit(
        train_generator,
        steps_per_epoch=1569,
        epochs=50,
        validation_data=test_generator,
        validation_steps=387)
classifier.save_weights('/content/gdrive/My Drive/Google Colab Folder/first_try.h5')  # always save your weights after training or during training

Epoch 1/50
  40/1569 [..............................] - ETA: 4:45 - loss: 8.4483 - accuracy: 0.4980

UnknownError: ignored

In [ ]:

model.fit (train_datagen, test_datagen, epochs =100, batch_size = batch_size,)

In [9]:
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.utils import array_to_img, img_to_array, load_img

train_datagen = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True
)

test_datagen = ImageDataGenerator(
            rescale = 1./255,
            shear_range = 0.2,
            zoom_range = 0.2,
            horizontal_flip = True
)

In [22]:
mypath2 = '/content/gdrive/My Drive/Google Colab Folder/train/'
img = load_img(str(mypath2) + 'car.1.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

i = 0
for batch in train_datagen.flow(x, batch_size=1,
                          save_to_dir='/content/gdrive/My Drive/Google Colab Folder/preview/', save_prefix='car', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

### Step 10: Create a training set from the training set folder.
'training_set' is the folder where our data has been placed. Our CNN model has an image size of `64x64`, so the same size should be passed here too. `batch_size` is the number of images in a single batch, which is `32`. `Class_mode` is set to binary since we are working on binary classifiers

### Step 11: Repeat step 10 for the test set 
while setting the folder to the location of the test images, that is, 'test_set'

### Step 12: Finally, fit the data. 
Set the `steps_per_epoch` to `STEP_SIZE_TRAIN` and the `validation_steps` to `STEP_SIZE_TEST`. 

Why do we need `steps_per_epoch` ?

Keep in mind that a Keras data generator is meant to loop infinitely — it should never return or exit.

Since the function is intended to loop infinitely, Keras has no ability to determine when one epoch starts and a new epoch begins.

Therefore, we compute the `steps_per_epoch` value as the total number of training data points divided by the batch size. Once Keras hits this step count it knows that it’s a new epoch.